In [ ]:
import os
import sys
import torch
import torch.backends.cudnn as cudnn
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from os import path
import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from network.TorchUtils import TorchModel
from features_loader import FeaturesLoaderVal
from utils.utils import register_logger, get_torch_device

In [ ]:
!nvidia-smi

# Definitions

In [ ]:
features_path = ''  # path to features
annotation_path = 'Test_Annotation.txt'  # path to annotations
model_path = './exps/model.weights'  # set logging file

# enable cudnn tune
cudnn.benchmark = True
device = get_torch_device()  # will use GPU if available, CPU otherwise

# Load dataset and detection model

In [ ]:
data_loader = FeaturesLoaderVal(features_path=args.features_path,
                                    annotation_path=args.annotation_path)
data_iter = torch.utils.data.DataLoader(data_loader,
                                            batch_size=1,
                                            shuffle=False,
                                            num_workers=0,  # 4, # change this part accordingly
                                            pin_memory=True)

model = TorchModel.load_model(args.model_path).to(device).eval()

# Predict scores

In [ ]:
y_trues = torch.tensor([])
y_preds = torch.tensor([])

with torch.no_grad():
    for features, start_end_couples, lengths in tqdm(data_iter):
        # features is a batch where each item is a tensor of 32 4096D features
        features = features.to(device)
        outputs = model(features).squeeze(-1)  # (batch_size, 32)
        for vid_len, couples, output in zip(lengths, start_end_couples, outputs.cpu().numpy()):
            y_true = np.zeros(vid_len)
            y_pred = np.zeros(vid_len)

            segments_len = vid_len // 32
            for couple in couples:
                if couple[0] != -1:
                    y_true[couple[0]: couple[1]] = 1

            for i in range(32):
                segment_start_frame = i * segments_len
                segment_end_frame = (i + 1) * segments_len
                y_pred[segment_start_frame: segment_end_frame] = output[i]

            if y_trues is None:
                y_trues = y_true
                y_preds = y_pred
            else:
                y_trues = np.concatenate([y_trues, y_true])
                y_preds = np.concatenate([y_preds, y_pred])

## Calculate ROC-AUC and plot the graph

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true=y_trues, y_score=y_preds, pos_label=1)

plt.figure()
lw = 2
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

os.makedirs('graphs', exist_ok=True)
plt.savefig(path.join('graphs', 'roc_auc.png'))
plt.close()
print('ROC curve (area = %0.2f)' % roc_auc)
